In [46]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Pre-requisites:
    1. Define seperate lists of independent search words like 'killed' and 'injured ' will have two different lists.
    2. expand list by using nltk synonyms functionality and try to ensure thorough list.
       To add : checks on grammar and rootword --Improvement required
    3. define function that will clean the data to remove any hyphen between words and other cleaning if required
Algorithm:
    1. fetch sentence with word present(preferable)
    2. set dependency tree and convert to networkx
    3. shortest path with condition thatit should not traverse node of other list
    4. handle case with multiple values

In [47]:
import nltk
nltk.download('words')
from nltk.corpus import words
from nltk.corpus import wordnet as wn
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import matplotlib as plt
from itertools import chain
from nltk import Tree
import networkx as nx
import pandas as pd

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Rishab\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [48]:
dictionary = list(dict.fromkeys(words.words()))

In [97]:
def get_synonyms(word):
    synonym =[]
    for syn_word in wn.synsets(word):
        for sw in syn_word.lemmas():
            synonym.append(sw.name())
    return list(synonym)

# def get_verb_tenses(word):

In [164]:
def listfetch(wordyouaresearchingfor):
    return list(set(get_synonyms(wordyouaresearchingfor) +[wordyouaresearchingfor]))
    

In [533]:
#temporary searches


killed_set = list(set(get_synonyms("killed") +["killed"] + ["claimed"]))
died = list(set(get_synonyms("died") + ["died"]))
injured_set = list(set(get_synonyms("injured") +["injured"]))
homeless_set = list(set(get_synonyms("homeless") +["homeless"]))
destroyed_set = list(set(get_synonyms("destroyed") +["destroyed"]))
displaced_set = list(set(get_synonyms("displaced") +["displaced"] +["drowned"] + ['displacement']))
pressure_set = list(set(get_synonyms("pressure") +["pressure"]))
speed_set = list(set(get_synonyms("speed") +["speed"] + ['Category']))

In [534]:
from itertools import chain
def merge_lists(list1,*args):
    return list1 + list(chain(*args))

other_list= merge_lists(injured_set,homeless_set,destroyed_set,displaced_set,pressure_set,speed_set)
died_set = merge_lists(killed_set,died)
other_list_lower = list(map(str.lower,other_list))
died_set_lower = list(map(str.lower,died_set))

yes


In [51]:
temp = []
for small_list in other_list:
    temp += small_list
newlist = temp

In [52]:
#add condition if required
#noticed in encyclopedia
def dict_check(word):
    if '-' in word:
        word_raw = word.replace('-', '')
        word_rep = wordnet_lemmatizer.lemmatize(word_raw)
        if word_rep in dictionary:
            return word_raw
        else:
            return word

In [53]:
dict_check("hur-ricane")

'hurricane'

In [54]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [55]:
def fetch_graph(sentence):
    edges = []
    doc = nlp(sentence)
    for token in doc:
        for child in token.children:
            edges.append(('{0}'.format(token.lower_),
                          '{0}'.format(child.lower_)))
    graph = nx.Graph(edges)
    return graph

In [56]:
def fetch_label(sentence):
    doc = nlp(sentence)
    cardinal_list={}
    for i,ent in enumerate(doc.ents):
        cardinal_list.update({ent.text:ent.label_})
    return cardinal_list

In [586]:
def other_entity_fetch(sentence):
    doc = nlp(sentence)
    entity_dict={}
    for i,ent in enumerate(doc.ents):
        entity_dict.update({ent.label_:ent.text})
        
    return entity_dict

In [304]:
def cardinal_check(dict1,dict2):
    card_dict_dict = {}
    cardinal_word_dict = []
    final_dict ={}

    for (key1,value1) in dict1.items():
        print(key1,value1)
        for (key2,value3) in dict2.items():
            print(key2,value3)
            for (word,value2) in value1.items():
                print(word,value2)
            
                if word == key2 and value3 == 'CARDINAL':
                    cardinal_word_dict.append({word:value2})
                     
                else:
                    pass
        resp_list = cardinal_word_dict
        card_dict_dict.update({key1:resp_list})
        print(card_dict_dict)
        cardinal_word_dict.clear()
        print(card_dict_dict)
    return card_dict_dict


In [306]:
sentence =  " three people died and 500 were injured due to a hurricane of Category 2 "
sen = sentence.lower()
c = fetch_label(sen)
otherwordlist_dict = shortestpath(graph,sentence,other_list_lower)
cardinal_check(otherwordlist_dict,c)

In [574]:
def fetch_cardinal(sentence):
    doc = nlp(sentence)
    cardinal_list=[]
    for i,ent in enumerate(doc.ents):
        if ent.label_ == 'CARDINAL':
            cardinal_list.append(ent.text)
    return cardinal_list

In [575]:
fetch_cardinal(sentence)

['100,000', '134']

In [571]:
# sentence
cardinal_list=[]
doc = nlp(sentence)
for i,ent in enumerate(doc.ents):
    if ent.label_ == 'CARDINAL':
        cardinal_list.append(ent.text)
    else:
        pass


In [572]:
(cardinal_list)

['100,000', '134']

In [134]:
def word_availability_check(sentence, searchlist): 
    available_list =[]
    s = sentence.split(" ")
    sentence = list(map(str.lower,s))
    for word in sentence:
        for searchword in searchlist:
            if (searchword == word):
                available_list.append(searchword)
#             elif(searchword == # can try lemma and tokenization after this if required
            elif(searchword == dict_check(word)):
                available_list.append(searchword)
            elif(searchword == wordnet_lemmatizer.lemmatize(word)):
                available_list.append(word)
            elif(wordnet_lemmatizer.lemmatize(searchword) == word):
                available_list.append(word)
            else:
                pass
    return list(set(available_list))
#returns list of available words

In [150]:
def shortestpath(graph,sentence,searchlist):
    spath_val = {}
    availablelist = word_availability_check(sentence,searchlist)
    for searchword in availablelist:
        next_word_dict = nx.single_source_shortest_path_length(graph,searchword)
        spath_val.update({searchword:next_word_dict})
    return spath_val
#returns dictionary with key and shortest path value from source

In [341]:
def shortestpath_text(graph,sentence,searchlist):
    spath_val = []
    
    availablelist = word_availability_check(sentence,searchlist)
    cardinal_list = fetch_cardinal(sentence)
    for searchword in availablelist:
#         print(searchword)
        for cardinal_word in cardinal_list:
#             print(cardinal_word)
            next_word_dict = nx.shortest_path(graph,searchword,cardinal_word)
#             print(next_word_dict)
            spath_val.append(next_word_dict)
            
    return spath_val

In [470]:
def length_shortest_path(spath_val_output):
    length_get = []
    
    for l in spath_val_output:
        
        length_get.append((l[0],l[-1],len(l)))
    
    return length_get

In [507]:
import itertools
import operator

def accumulate(l):
    it = itertools.groupby(l, operator.itemgetter(0))
    for key,subiter in it:
       yield key, min(item[1] for item in subiter)

# list(accumulate(a))
minValue = [i for i in a if i[1] == min(x[1] for x in a)]
minValue
Output = ([min(c) for a, b in itertools.groupby(Input, lambda y: y[0])]) 

In [530]:
def info_fetch(sentence,searchlist,otherwordlist):
    fetch_cardinal = {}
    sentence = sentence.lower()
    doc = nlp(sentence)
    tree = [to_nltk_tree(sent.root) for sent in doc.sents]
    print([to_nltk_tree(sent.root).pretty_print() for sent in doc.sents])
    graph = fetch_graph(sentence)
    context_dict = fetch_label(sentence)
    spath_val_output_diedset = shortestpath_text(graph,sentence,died_set_lower)
    l1 = length_shortest_path(spath_val_output_diedset)
    spath_val_output_otherset = shortestpath_text(graph,sentence,other_list_lower)
    l2 = length_shortest_path(spath_val_output_otherset)
    final_list = l1+l2
    df = pd.DataFrame(final_list)
    info = df.loc[df.groupby(1)[2].idxmin()]

    return info

#     search_path_dict = shortestpath(graph,sentence,searchlist)
#     otherwordlist_dict = shortestpath(graph,sentence,otherwordlist)    
#     search_path_dict_filtered = cardinal_check(search_path_dict,context_dict)
#     otherwordlist_dict_filtered = cardinal_check(otherwordlist_dict,context_dict)

In [594]:
sentence = "During a five-day sojourn up the eastern \
seaboard in June 1972, Agnes forced the displacement \
of 100,000 people, caused nearly \
$2 billion in flood damage, and claimed 134 lives in\
Florida, Georgia, Pennsylvania, New York, Virginia,\
and Maryland."
df1 = info_fetch(sentence,died_set_lower,other_list_lower)
df1

                                                                                              forced                                                                                                    
  ______________________________________________________________________________________________|__________                                                                                              
 |    |    |   |                      |                                       |                          caused                                                                                         
 |    |    |   |                      |                                       |                  __________|______________________________                                                               
 |    |    |   |                      |                                       |                 |     |            |                   claimed                                                    

,0,1,2
2,displacement,"100,000",4
1,claimed,134,3


In [595]:
other_entity = other_entity_fetch(sentence)
df2 = pd.DataFrame.from_dict(other_entity,orient='index')
df2.T

,DATE,ORG,CARDINAL,MONEY,GPE
0,June 1972,Agnes,134,nearly $2 billion,Maryland


In [596]:
sentence = "Two people were killed and 15 were injured in Haikou"
df1 = info_fetch(sentence,died_set_lower,other_list_lower)
df1

         killed                   
  _________|___________            
 |    |    |        injured       
 |    |    |      _____|______     
 |    |  people  |     |      in  
 |    |    |     |     |      |    
were and  two    15   were  haikou

[None]


,0,1,2
3,injured,15,2
0,killed,two,3


In [597]:
other_entity = other_entity_fetch(sentence)
df2 = pd.DataFrame.from_dict(other_entity,orient='index')
df2.T

,CARDINAL,GPE
0,15,Haikou


### DEBUG PORTION

In [552]:
fetch_cardinal = {}
sentence = sentence.lower()
doc = nlp(sentence)
tree = [to_nltk_tree(sent.root) for sent in doc.sents]
print([to_nltk_tree(sent.root).pretty_print() for sent in doc.sents])


                                                                                              forced                                                                                                  
  ______________________________________________________________________________________________|_______________                                                                                       
 |    |    |   |                      |                                       |                               caused                                                                                  
 |    |    |   |                      |                                       |                        _________|____________________________                                                          
 |    |    |   |                      |                                       |                       |   |              |                claimed                                                     
 | 

In [577]:
graph = fetch_graph(sentence)
context_dict = fetch_label(sentence)
spath_val_output_diedset = shortestpath_text(graph,sentence,died_set_lower)


In [ ]:
l1 = length_shortest_path(spath_val_output_diedset)
spath_val_output_otherset = shortestpath_text(graph,sentence,other_list_lower)
l2 = length_shortest_path(spath_val_output_otherset)
final_list = l1+l2
df = pd.DataFrame(final_list)
info = df.loc[df.groupby(1)[2].idxmin()]

In [551]:
availablelist = word_availability_check(sentence,other_list_lower)
availablelist

['displacement']

{'500': 'CARDINAL', 'Category 2': 'PRODUCT', 'three': 'CARDINAL'}

In [ ]:
# # def cardinal_check(dict1,dict2):
# #     card_dict_dict = {}
# #     cardinal_word_dict = {}
# #     final_dict ={}

# #     for (key1,value1) in dict1.items():
# #         for (word,value2) in value1.items():
# #             for (key2,value3) in dict2.items():
# #                 if word == key2 and value3 == 'CARDINAL':
# # #                     print('---', word, value2)
# #                     cardinal_word_dict.update({word:value2})
# #                 else:
# #                     pass
# #             print('--  ---', cardinal_word_dict)
# #             card_dict_dict.update({key1:cardinal_word_dict})
# # #             cardinal_word_dict.clear()
                
        
                
            
# #     return dict1
# # def cardinal_check(dict1,dict2):
# #     card_dict_dict = {}
# #     cardinal_word_dict = {}
# #     final_dict ={}

# #     for (key1,value1) in dict1.items():
# #         for (word,value2) in value1.items():
# #             for (key2,value3) in dict2.items():
# #                 if word == key2 and value3 != 'CARDINAL':
# # #                     print('---', word, value2)
# #                     value1.pop(word)
# #                 else:
# #                     pass
# #         card_dict_dict.update({key1:value1})
                
                
        
                
            
# #     return card_dict_dict
# # newDict = dict(filter(lambda elem: {elem[0],elem[1]} if word == key2 and value3 == 'CARDINAL', value1.items() for value1))
# # cardinal_dict = dict(filter(lambda elem:(elem[0],elem[1]) if (elem[0] == key2 and value3 == 'CARDINAL' else None, value1.items()))
# #     for k, v in dict1.items():
# #         for k_, v_ in v.items():
# #             if v_ in dict2.keys() and dict2[v_] == 'CARDINAL':
# #                 final_dict[k_] = 
# #                 continue
# #             else:
# #                 del dict1[k][k_]

# # def get_closest_value(dict1,dict2):
# #     for k1,v1 in dict1.items():
# #         for k2,v2 in dict2.items():
# #             if v1>v2:
# #                 dict1.pop(k1)
# #             elif v2<v1:
# #                 dict2.pop(k2)
                
# # # word_val = []
# # # for l in final_list:
# # #     print(l)
# # #     for x in final_list:
# # #         print(x)
# # #         if l[1] == x[1] and x[2]>l[2]:
# # #             print(l)
# # minValue = [i[0] for i in final_list if i[1] == min(x[1] for x in final_list)]
# # # min(final_list, key=lambda t: (t[0],t[1],t[2]))
# # minValue     

# def get_closest_path(searchlist,otherlist):
#     path_val =[]
#     for l in searchlist:
#         print(l)
#         if len(l)==2:
#             path_val.append((l[0],l[1],2))
#             print(path_val)
# #         elif len(l)>2:
# #             for word in otherlist:
# #                 print(word)
# #                 if word in l:
# #                     windex = l.index(word)
# #                     my_list = l[windex:]
# #                     print(my_list)
# #                     if len(my_list)==2:
# #                         path_val.append((my_list[0],my_list[1],2))
# #                     else:
# #                         path_val.append((my_list[0],my_list[1],len(my_list)))
#         else:
#             pass
#         return path_val